In [1]:
import os
from pathlib import Path
import pandas as pd
# import numpy as np
from sqlalchemy import create_engine,text
# from scipy import stats
import plotly.express as px
import toml
import psrc_theme

# %matplotlib inline
# from IPython.display import display, HTML

# to show plotly figures in quarto HTML file
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"
pio.templates.default = "simple_white+psrc_color" # set plotly template

input_config = toml.load(os.path.join(os.getcwd(), '..\\..\\..\\..\\configuration', 'input_configuration.toml'))
valid_config = toml.load(os.path.join(os.getcwd(), '..\\..\\..\\..\\configuration', 'validation_configuration.toml'))
# emme_config = toml.load(os.path.join(os.getcwd(), '..\\..\\..\\..\\configuration','emme_configuration.toml'))
# network_config = toml.load(os.path.join(os.getcwd(), '..\\..\\..\\..\\configuration','network_configuration.toml'))

# table format
pd.options.display.float_format = '{:0,.0f}'.format
format_percent = "{:,.1%}".format

In [2]:
# summary data location
df_result = pd.read_csv(Path(valid_config['model_dir']) / 'outputs/network/network_results.csv')

# observed counts location
conn = create_engine('sqlite:///../../../../inputs/db/'+input_config['db_name'])
df_observed_bike = pd.read_sql(text("SELECT * FROM observed_bike_volumes"), con=conn.connect())

In [3]:
df_observed_bike.columns

Index(['Unnamed: 0', 'CountID', 'Counter ID', 'Jurisdiction', 'County',
       'Agency', 'Follow Up Notes', 'Directions', 'OBJECT ID', 'PSRCEdgeID',
       'PSRCJunctID', 'CrossStreets', 'Latitude', 'Longitude', 'Year',
       'CountDate', 'P_1', 'P_2', 'P_3', 'P_4', 'P_D', 'B_1', 'B_2', 'B_3',
       'B_4', 'B_D', 'P_IJ_1', 'P_IJ_2', 'P_IJ_3', 'P_IJ_4', 'P_IJ_D',
       'B_IJ_1', 'B_IJ_2', 'B_IJ_3', 'B_IJ_4', 'B_IJ_D', 'P_JI_1', 'P_JI_2',
       'P_JI_3', 'P_JI_4', 'P_JI_D', 'B_JI_1', 'B_JI_2', 'B_JI_3', 'B_JI_4',
       'B_JI_D'],
      dtype='object')

In [4]:
tod_map = {'B_IJ_1': '6to7',
           'B_JI_1': '6to7',
           'B_IJ_2': '7to8',
           'B_JI_2': '7to8',
           'B_IJ_3': '16to17',
           'B_JI_3': '16to17',
           'B_IJ_4': '17to18',
           'B_JI_4': '17to18'}

## top bike links

In [5]:
# Group both directions
# _df = df[df['@countid'].isin(df_obs['CountID'])]
df = df_result.groupby(['tod','@countid'])['@bvol'].sum()
df = df.reset_index()
result = df[df['tod'].isin(tod_map.values())].sort_values('@bvol',ascending=False)

result.iloc[:9]


,tod,@countid,@bvol
9384,16to17,0,"669,757"
12512,17to18,0,"662,630"
28145,7to8,0,"580,835"
25017,6to7,0,"422,409"
10825,16to17,"2,045",488
13954,17to18,"2,046",471
15394,17to18,"6,001",471
29587,7to8,"2,046",444
31027,7to8,"6,001",444
